In [3]:
import pandas as pd

def read_clean_merge_suicide_2016_2022(fname, varname):
    print(f"Reading, cleaning, and merging data from: {fname} ...")
    print(f"Using var: {varname}")
    df = pd.read_csv(fname, sep=";")

    print("Extracting dpto and municipio ...")
    df["DPTO"] = df["Municipio"].apply(lambda x: x.split()[0])
    df["MUNICIPIO"] = df["Municipio"].apply(lambda x: x.split('/')[-1])
    df.drop(columns="Municipio", inplace=True)

    print("Renaming columns ...")
    df.rename(columns={"Año":"YEAR", "Subcategoria":varname, "Sexo":"GENDER", "Valor":"SUI_COUNTER"}, inplace=True)

    print("Normalizing names ...")
    df["DPTO"] = df["DPTO"].str.upper().str.normalize("NFKD").str.encode('ascii', errors='ignore').str.decode('utf-8').str.strip()
    df["MUNICIPIO"] = df["MUNICIPIO"].str.upper().str.normalize("NFKD").str.encode('ascii', errors='ignore').str.decode('utf-8').str.strip()
    df["GENDER"] = df["GENDER"].str.replace('Hombre', 'MASCULINO') 
    df["GENDER"] = df["GENDER"].str.replace('Mujer', 'FEMENINO') 

    print("Fixing San Andres and Valle del Cauca and Bogota names ...")
    df["DPTO"].replace({"VALLE DEL CAUCA":"VALLE", 
                        "ARCHIPIELAGO DE SAN ANDRES, PROVIDENCIA Y SANTA CATALINA":"SAN ANDRES",
                        "BOGOTA, D.C.":"BOGOTA"}, 
                        inplace=True)
    df["MUNICIPIO"].replace({"BOGOTA, D.C.":"BOGOTA"}, inplace=True)
    df.loc[df["MUNICIPIO"].str.contains("BOGOTA"), ["DPTO", "MUNICIPIO"]] = "BOGOTA"
    df.loc[df["DPTO"].str.contains("BOGOTA"), ["DPTO", "MUNICIPIO"]] = "BOGOTA"
    #print(df[df["MUNICIPIO"].str.contains("BOG")])

    print("Changing year format to datetime ...")
    df['YEAR'] = pd.to_datetime(df['YEAR'], format="%Y").dt.to_period('Y')

    print("Removing data with no data points ...")
    df = df.drop(df[(df["GENDER"] == "Indeterminado") & (df["SUI_COUNTER"] == 0)].index)

    print("Loading geo data ...")
    dfgeo = pd.read_csv("INPUTDATA/CSV/CodigosDivisionPolitica.csv")
    #dfgeo.head()

    print("Merging with geo data and saving ...")
    df = df.merge(dfgeo, how="inner", on=["DPTO", "MUNICIPIO"], indicator=False)
    #df.tail(10)
    df.to_csv(f"INPUTDATA/NEWCSV/MERGED-2016-2022-Suicidios-Geo-{varname}.csv", index=False)
    #df.head()

    print("To merge with population, grouping to remove gender since population data has no gender ... ")
    df = df.groupby(by=["YEAR", "DPTO", "CODE_DPTO", "MUNICIPIO", "CODE_MUNICIPIO",  "LONGITUD", "LATITUD"]).sum().reset_index()
    dfpop = pd.read_csv("INPUTDATA/CSV/Population-2016-2022.csv")
    dfpop['YEAR'] = pd.to_datetime(dfpop['YEAR'], format="%Y").dt.to_period('Y')
    df = df.merge(dfpop, how="inner", on=["DPTO", "MUNICIPIO", "CODE_DPTO", "CODE_MUNICIPIO", "YEAR"], indicator=False)
    df.to_csv(f"INPUTDATA/NEWCSV/MERGED-2016-2022-Suicidios-Geo-Pop-{varname}.csv", index=False)
    
    print("Done.")


In [10]:
fnames_vars = {"INPUTDATA/CSV/CONSOLIDADO RAZON DEL SUICIDIO.csv":"CAUSE", 
               "INPUTDATA/CSV/CONSOLIDADO ESCOLARIDAD.csv":"SCHOLARSHIP",
               "INPUTDATA/CSV/CONSOLIDADO GRUPO DE EDAD.csv":"AGE_GROUP", 
               "INPUTDATA/CSV/CONSOLIDADO ANCESTRO RACIAL.csv":"RACIAL_ORIGIN",
               "INPUTDATA/CSV/CONSOLIDADO ESTADO CONYUGAL.csv":"MARITAL_STATUS",
               "INPUTDATA/CSV/CONSOLIDADO FACTOR DE VULNERABILIDAD.csv":"VULNERABILITY_FACTOR"
              }
for fname, varname in fnames_vars.items():
    #print(fname, varname)
    read_clean_merge_suicide_2016_2022(fname, varname)

Reading, cleaning, and merging data from: INPUTDATA/CSV/CONSOLIDADO RAZON DEL SUICIDIO.csv ...
Using var: CAUSE
Extracting dpto and municipio ...
Renaming columns ...
Normalizing names ...
Fixing San Andres and Valle del Cauca and Bogota names ...
Changing year format to datetime ...
Removing data with no data points ...
Loading geo data ...
Merging with geo data and saving ...
To merge with population, grouping to remove gender since population data has no gender ... 
Done.
Reading, cleaning, and merging data from: INPUTDATA/CSV/CONSOLIDADO ESCOLARIDAD.csv ...
Using var: SCHOLARSHIP
Extracting dpto and municipio ...
Renaming columns ...
Normalizing names ...
Fixing San Andres and Valle del Cauca and Bogota names ...
Changing year format to datetime ...
Removing data with no data points ...
Loading geo data ...
Merging with geo data and saving ...
To merge with population, grouping to remove gender since population data has no gender ... 
Done.
Reading, cleaning, and merging data from: